# Blast Application - By Ethan

In [2]:
# Imports
from Bio.Blast import NCBIWWW, NCBIXML
from Bio import SeqIO
import pandas as pd
from lxml import etree as et
import numpy as np

Using the previously made command line script, we can write a function that inputs a file name (including the file extension) and run a blast

In [49]:
def blast(file_name: str, blast_type: str, max_hits: int = None, megablast: bool = False, e_value_threshold: float = None):
    """
    A function that runs a BLAST of desired type and saves the results as an xml file. 
    :inputs: file_name (including file extension), blast_type (one of blastn, blastp, tblastn or tblastx)
    :returns: None
    """ 

    # Error check
    if blast_type.lower() in ["blastn", "blastp", "blastx", "tblastn", "tblastx"]:
        
        # Open sequence file
        seq_file = next(SeqIO.parse(open(file_name), "fasta"))

        # Megablast
        if blast_type == "blastn":
            if megablast == True:
                query = NCBIWWW.qblast(blast_type, "nt", seq_file.seq, megablast=True)

        # Start query
        print("Starting BLAST query on NCBI database...")
        print("Please hold. This process can take up to 10 minutes to complete...")
        query = NCBIWWW.qblast(blast_type, "nt", seq_file.seq)
        print(query)

        # Store results as XML
        print("Storing results as an XML file...")
        with open(str(file_name).split(".")[0] + "_results.xml", "w") as save_file:
            blast_results = query.read()
            save_file.write(blast_results)

        print("Success! " + str(file_name).split(".")[0] + "_results.xml has been saved!")
        
        # Make raw dataframe
        print("Parsing XML file as dataframe...")
        df = pd.read_xml(str(file_name).split(".")[0] + "_results.xml", xpath=".//Hsp")
        
        # Edit raw dataframe
        # Rename and drop columns
        df = df.drop(["Hsp_num", "Hsp_qseq", "Hsp_query-frame", "Hsp_hit-frame"], axis=1)
        df = df.rename(columns={"Hsp_bit-score" : "bit_score", "Hsp_score" : "score", "Hsp_evalue" : "evalue", "Hsp_query-from" : "query_from", "Hsp_query-to" : "query_to",
        "Hsp_hit-from" : "hit_from", "Hsp_hit-to" : "hit_to", "Hsp_identity" : "identity", "Hsp_positive" : "positive", "Hsp_gaps" : "gaps", "Hsp_align-len" : "align_len",
        "Hsp_hseq" : "hit_seq", "Hsp_midline" : "midline"})

        # Add new columns based on Hit XML alltributes
        tree = et.parse(open(str(file_name).split(".")[0] + "_results.xml"))
        hit_id = tree.xpath(".//Hit/Hit_id/text()")
        hit_def = tree.xpath(".//Hit/Hit_def/text()")
        hit_accession = tree.xpath(".//Hit/Hit_accession/text()")
        hit_length = tree.xpath(".//Hit/Hit_len/text()")
        df["id"] = hit_id
        df["description"] = hit_def
        df["accession"] = hit_accession
        df["acc_len"] = hit_length

        # Calculating percent identity
        df["per_ident"] = np.round((df["identity"] / df["align_len"])*100, decimals=2)

        # Calculating query coverage
        df["query_cover"] = np.round((df["query_to"] - df["query_from"] + 1)/len(seq_file.seq)*100, decimals=2)

        # Change order of columns
        df = df[["id", "description", "bit_score", "score", "query_cover", "per_ident", "evalue", "query_from", "query_to", 
        "hit_from", "hit_to", "identity", "positive", "gaps", "align_len", "acc_len", "hit_seq", "midline", "accession"]]

        # Remove rows if e-value is over threshold
        if e_value_threshold is not None:
            df = df[df.evalue < e_value_threshold]
        
        # Cap rows/hits if there is a maximum
        if max_hits is not None:
            df = df.sort_values(by="evalue", ascending=False)
            df = df.head(max_hits)

        # Save editted dataframe
        print("Saving dataframe as csv file...")
        df.to_csv(str(file_name).split(".")[0] + "_dataframe.csv")

        # End
        print("Success! Dataframe has successfully been saved as " + str(file_name).split(".")[0] + "_dataframe.csv" + ". Press any key to exit function.")
    else:
        print("blast_type not one of blastn, blastp, blastx, tblastn or tblastx")
        blast(file_name=file_name, blast_type=blast_type, max_hits=max_hits, megablast=megablast, e_value_threshold=e_value_threshold)   

Running a test of the function:

In [50]:
blast(file_name="aporrectodea_rosea.fna", blast_type="blastn", max_hits=5, megablast=False, e_value_threshold=1e-20)

Starting BLAST query on NCBI database...
Please hold. This process can take up to 10 minutes to complete...
Storing results as an XML file...
Success! aporrectodea_rosea_results.xml has been saved!
Parsing XML file as dataframe...
Saving dataframe as csv file...
Success! Dataframe has successfully been saved as aporrectodea_rosea_dataframe.csv. Press any key to exit function.


Processing the dataframe. Here is the csv:

In [52]:
df = pd.read_csv("aporrectodea_rosea_dataframe.csv")
df

,Unnamed: 0,id,description,bit_score,score,query_cover,per_ident,evalue,query_from,query_to,hit_from,hit_to,identity,positive,gaps,align_len,acc_len,hit_seq,midline,accession
0,0,gi|1829765830|ref|NC_046733.1|,Aporrectodea rosea haplogroup L4 mitochondrion...,2778.47,3080,100.00,100.00,0,1,1540,1,1540,1540,1540,0,1540,15086,ATGCGATGATTTTACTCAACCAATCACAAAGATATTGGAACTTTAT...,||||||||||||||||||||||||||||||||||||||||||||||...,NC_046733
1,37,gi|1068288817|gb|KU728851.1|,Enchytraeus cf. crypticus SL-2017 cytochrome c...,1349.30,1495,99.09,79.62,0,1,1526,1,1523,1215,1215,3,1526,1539,ATGCGCTGACTATATTCTACAAACCACAAAGACATTGGTACACTAT...,||||| ||| | || || || || |||||||| ||||| || |||...,KU728851
2,27,gi|1003312217|gb|KT429019.1|,"Amynthas robustus mitochondrion, complete genome",1387.18,1537,99.74,80.01,0,1,1536,1,1536,1229,1229,0,1536,15013,ATGCGATGATTATATTCTACAAACCACAAAGACATTGGAACCCTAT...,||||||||||| || || || || |||||||| |||||||| |||...,KT429019
3,28,gi|827022294|gb|KP688582.1|,"Amynthas gracilis mitochondrion, complete genome",1381.77,1531,99.22,80.04,0,1,1528,1,1528,1223,1223,0,1528,15161,ATGCGATGATTGTATTCTACAAACCACAAAGACATTGGAACCCTAT...,||||||||||| || || || || |||||||| |||||||| |||...,KP688582
4,29,gi|1003312161|gb|KT429015.1|,"Duplodicodrilus schmardae mitochondrion, compl...",1379.96,1529,99.81,79.90,0,1,1537,1,1537,1228,1228,0,1537,15156,ATGCGATGATTATATTCTACAAATCACAAAGACATTGGAACTTTAT...,||||||||||| || || || ||||||||||| |||||||||||||...,KT429015


Testing BLAST multiple:

In [3]:
def blast_multiple(folder_directory: str, blast_type: str, max_hits: int = None, megablast: bool = False, e_value_threshold: float = None):
    """
    A function that runs multiple BLASTs of a desired type and saves the results as a csv file.
    :inputs: folder_directory, blast_type (one of blastn, blastp, tblastn or tblastx), 
            max_hits, megablast, e_value_threshold
    :returns: None
    """ 

    # Imports
    from Bio.Blast import NCBIWWW, NCBIXML
    from Bio import SeqIO
    import pandas as pd
    from lxml import etree as et
    import numpy as np
    from blast import blast

    # Get each file
    file_names = []
    count = 0
    for filename in os.listdir(folder_directory):
        f = os.path.join(folder_directory, filename)
        # Check if the file exists
        if os.path.isfile(f) and filename.split(".")[-1] == "fna":
            count += 1
            print("Searching " + folder_directory + ". Found " + str(count) + " sequences to BLAST so far.", end="\r")
            file_names.append(str(f))

    for file_name in file_names:
        count = 0
        print("Currenting BLASTing " + str(file_name) + ". Have BLASTed " + str(count) + " sequences so far.", end="\r")
        blast(file_name=file_name, blast_type=blast_type, max_hits=max_hits, megablast=megablast, e_value_threshold=e_value_threshold)
    print("Success! BlASTed a total of " + str(count) + " sequences.")

    return

In [4]:
blast_multiple(folder_directory = "to_blast", blast_type="blastn", max_hits= 5, megablast=False, e_value_threshold=1e-20)

Starting BLAST query on NCBI database...riensis.fna. Have BLASTed 0 sequences so far.
Please hold. This process can take up to 10 minutes to complete...
Storing results as an XML file...
Success! to_blast\amynthas_jiriensis_results.xml has been saved!
Parsing XML file as dataframe...
Saving dataframe as csv file...
Success! Dataframe has successfully been saved as to_blast\amynthas_jiriensis_dataframe.csv. Press any key to exit function.
Starting BLAST query on NCBI database...ungpanensis.fna. Have BLASTed 0 sequences so far.
Please hold. This process can take up to 10 minutes to complete...
Storing results as an XML file...
Success! to_blast\amynthas_seungpanensis_results.xml has been saved!
Parsing XML file as dataframe...
Saving dataframe as csv file...
Success! Dataframe has successfully been saved as to_blast\amynthas_seungpanensis_dataframe.csv. Press any key to exit function.
Starting BLAST query on NCBI database...a_rosea.fna. Have BLASTed 0 sequences so far.
Please hold. This 

Now to work on a sequence downloader

In [2]:
# Imports
from Bio import Entrez, SeqIO
import time

In [7]:
def download(file_name: str, email_address: str, gene_name: str):
    """"
    A function that searches the genbank gene database and downloads multiple fasta sequence files
    :inputs: file_name (including file extension) of a text file containing line separated species names to download sequences for,
            email_address so NCBI knows who is requesting files data if anything goes awry,
            gene_name containing the gene of interest to search for
    :returns: None
    """

    # Imports
    from Bio import Entrez, SeqIO
    import time

    # Parameters
    Entrez.api_key = "0479a2cd7f6d9e97e14c35e4d49eba018509"
    Entrez.email = email_address

    # Read text file
    with open(file_name) as f:
        species_names = [line.rstrip('\n') for line in f]

    # Start search
    print("Starting search. Please hold as only 10 API requests can be made per second...")
    record_ids = []
    record_counts = 0
    for species_name in species_names:
        handle = Entrez.esearch(db="nucleotide", term=species_name + "[Orgn] AND COI[Gene]", idtype="acc", retmax=5000)
        record = Entrez.read(handle)
        record_counts += int(record["Count"])
        record_ids += record["IdList"]
        print("Found " + str(record_counts) + " records so far...", end="\r")
        handle.close()

        # Wait
        time.sleep(1)
    print("Search success! For a search of " + str(len(species_names)) + " species, a total of " + str(record_counts) + " genbank records containing COI genes were found!")

    # Start download
    print("Starting sequence download process. Please hold as only 10 API requests can be made per second...")
    count = 0
    for record_id in record_ids:
        if not os.path.isfile("downloads//" + gene_name + "//" + str(record_id) + ".fna"):
            net_handle = Entrez.efetch(db="nucleotide", id=record_id, rettype="fasta", retmode="text", retmax=5000)
            out_handle = open("downloads//" + gene_name + "//" + str(record_id) + ".fna", "w")
            out_handle.write(net_handle.read())
            out_handle.close()
            net_handle.close()
            count += 1
            print("Downloading record " + str(record_id) + " Downloaded " + str(count) + " sequences so far...", end="\r")

            # Wait
            time.sleep(1)

    # End
    print("Success! You can find your downloaded sequences in the downloads folder where this script is located!")


Testing the download function:

In [8]:
download(file_name="nematode_names.txt", email_address="epay0001@student.monash.edu", gene_name="COI")

Starting search. Please hold as only 10 API requests can be made per second...
Search success! For a search of 82 species, a total of 2115 genbank records containing COI genes were found!
Starting sequence download process. Please hold as only 10 API requests can be made per second...
Success! You can find your downloaded sequences in the downloads folder where this script is located!


Now to write a function that checks for any duplicate sequences

In [4]:
def check_duplicates(folder_directory: str):
    """
    """
    
    # Imports
    from Bio import SeqIO
    import time

    # Loop through each file
    file_paths_to_remove = set()

    for filename_one in os.listdir(folder_directory):
        f_one = os.path.join(folder_directory, filename_one)

        # Check if the file exists
        if os.path.isfile(f_one) and filename_one.split(".")[-1] == "fna":
            
            with open(f_one) as handle_one:
                current_seq = next(SeqIO.parse(handle_one, "fasta"))
                # Check current_seq against every other file
                for filename_two in os.listdir(folder_directory):
                    f_two = os.path.join(folder_directory, filename_two)
                    
                    # Check if the file exists
                    if os.path.isfile(f_two) and filename_two.split(".")[-1] == "fna":
                        
                        # Check if the file isn't the same as the one we are checking
                        if filename_one != filename_two:
                            
                            with open(f_two) as handle_two:
                                check_seq = next(SeqIO.parse(handle_two, "fasta"))
                                
                                print("Comparing " + filename_one + " to " + filename_two, end="\r")
                                # Check if the sequences of the fasta files are the same:                     
                                if current_seq.seq == check_seq.seq:
                                    file_paths_to_remove.add(f_two)
            
            

    print("Found: " + str(len(file_paths_to_remove)) + " duplicate files to remove. Removing...")  
    
    for file_name in file_paths_to_remove:
        os.remove(file_name)
        # Wait
        time.sleep(1)
    print("Success!")
    

Testing the check_duplicates function:

In [5]:
check_duplicates(folder_directory = "downloads\COI")

Found: 1112 duplicate files to remove. Removing...
Success!
